In [ ]:
from time import sleep

import pandas as pd
import requests


In [ ]:
df = pd.DataFrame()
start = 0
dois = []

In [ ]:
#TO DO--MAKE REQUESTS RUN CONCURRENTLY & ADAPTABLE FOR BOTH MEDRXIV AND BIORXIV
#CROSSREF MAX QUERIES: 50 / SECOND
database = 'biorxiv'
start_date = '2019-01-01'
end_date = '2022-02-09'

In [ ]:
while True:
    try:
        r = requests.get(f'https://api.{database}.org/details/{database}/{start_date}/{end_date}/{start}', '')
    except:
        sleep(60)
        r = requests.get(f'https://api.{database}.org/details/{database}/{start_date}/{end_date}/{start}', '')
    j = r.json()
    r.close()
    if j['messages'][0]['status'] == 'no posts found': break
    for article in j['collection']:
        try:
            dois.append(article['doi'])
        except:
            pass
    start += 100
    #sleep(60)

    if start % 2 == 0:
        df = pd.DataFrame() 
        df['doi'] = dois
        df.to_csv(f'{database} doi data.csv')



In [ ]:
z = list(set(dois))

In [ ]:
df.to_csv(f'{database}_{start_date}-{end_date}_dois.csv')

In [ ]:
import queue
import threading
from time import sleep
import time


In [ ]:
df = pd.read_csv('')
df.drop_duplicates('doi', inplace=True)
temp = set(list(df['doi']))
df['first_names'] = ''
df['last_names'] = ''
df['title'] = ''
df.set_index('doi', inplace=True)

In [ ]:
df = pd.read_csv('/content/biorxiv_2019-2022_dois.csv')
df.drop_duplicates('doi', inplace=True)
temp = set(list(df['doi']))
df['first_names'] = ''
df['last_names'] = ''
df['title'] = ''
df.set_index('doi', inplace=True)

In [ ]:
df['first_names'] = ''
df['last_names'] = ''
df['title'] = ''

In [ ]:
df.head()

In [ ]:
first = {}
last = {}
titles = {}

for i in df.index:
    first[i] = []
    last[i] = []
    titles[i] = ''

In [ ]:
def get_authors():
    while True:
        try:
            doi = q.get(block=False)[0]
        except queue.Empty:
            break
        if df.loc[doi, 'first_names'] != '':
            print('repeat?')
            q.task_done
        else:
            _start = time.time()

            cross_ref(doi)

            _end = time.time()
            if _end - _start < 1:
                sleep(1 - (_end - _start))
            q.task_done()


In [ ]:
def cross_ref(url):
    _f = []
    _l = []
    r = requests.get(f'https://api.crossref.org/v1/works/{url}')
    if r.text == 'Resource not found.' or r.status_code != 200:
        print('not found: ', url)
        df.loc[url, 'title'] = ''
        return
    j = r.json()
    r.close()
    try:
        for author in j['message']['author']:
            _f.append(str(author['given']))
            _l.append(str(author['family']))
    except:
        pass

    titles[url] = j['message']['title'][0]
    first[url] = _f
    last[url] = _l
    df.loc[url, 'title'] = j['message']['title'][0]
    df.loc[url, 'first_names'] = f'{_f}'
    df.loc[url, 'last_names'] = f'{_l}'
    return

In [ ]:
NUM_THREADS = 40
threads = []

q = queue.Queue()

for d in temp:
    q.put([d])

for _ in range(NUM_THREADS):
    threads.append(threading.Thread(target=get_authors))

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

df.to_csv(f'{database}_with_names_2_11.csv')

In [ ]:
first_names = pd.DataFrame(first)
first_names.to_csv('biorxiv_first_names.csv')
last_names = pd.DataFrame(last)
last_names.to_csv('biorxiv_last_names.csv')
titless = pd.DataFrame(titles)
titless.to_csv('biorxiv_titles.csv')